In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import cv2  # OpenCV for image processing
import os
import random
import time
import joblib

In [7]:
class CrowSearchAlgorithm:
    def __init__(self, num_crows, max_iter, search_space, awareness=0.1, flight_length=2):
        self.num_crows = num_crows
        self.max_iter = max_iter
        self.search_space = search_space
        self.awareness = awareness
        self.flight_length = flight_length

    def fitness(self, params, X_train, y_train, X_val, y_val):
        # Train model with given parameters
        model = RandomForestClassifier(n_estimators=int(params[0]), max_depth=int(params[1]), random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        print(f"Current params: {params}, Validation accuracy: {accuracy:.4f}")
        return accuracy

    def optimize(self, X_train, y_train, X_val, y_val):
        print("Starting Crow Search Optimization...")
        crows_position = np.random.uniform(self.search_space[0], self.search_space[1], 
                                           (self.num_crows, len(self.search_space[0])))
        memory = crows_position.copy()
        best_solution = crows_position[0]
        best_fitness = self.fitness(best_solution, X_train, y_train, X_val, y_val)

        for t in range(self.max_iter):
            print(f"Iteration {t+1}/{self.max_iter}")
            for i in range(self.num_crows):
                if random.random() > self.awareness:
                    random_crow = random.randint(0, self.num_crows - 1)
                    crows_position[i] = crows_position[i] + self.flight_length * (memory[random_crow] - crows_position[i])
                    crows_position[i] = np.clip(crows_position[i], self.search_space[0], self.search_space[1])
                else:
                    crows_position[i] = np.random.uniform(self.search_space[0], self.search_space[1])

                current_fitness = self.fitness(crows_position[i], X_train, y_train, X_val, y_val)
                if current_fitness > self.fitness(memory[i], X_train, y_train, X_val, y_val):
                    memory[i] = crows_position[i]
                    if current_fitness > best_fitness:
                        best_fitness = current_fitness
                        best_solution = crows_position[i]

            print(f"Best solution so far: {best_solution}, Fitness: {best_fitness:.4f}")

        print("Optimization complete!")
        return best_solution, best_fitness


In [8]:
def load_and_preprocess_images(image_folder, target_size=(128, 128)):
    data = []
    labels = []
    label_map = {
        'Non Demented': 0,
        'Very mild Dementia': 1,
        'Mild Dementia': 2,
        'Moderate Dementia': 3
    }

    print("Loading and preprocessing images...")
    start_time = time.time()
    for label in label_map.keys():
        folder_path = os.path.join(image_folder, label)
        print(f"Processing images for class: {label}")
        iter=0
        for filename in os.listdir(folder_path):
            iter+=1
            if iter>500:
                continue
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, target_size)
            data.append(img.flatten())
            labels.append(label_map[label])
    print(f"Image loading and preprocessing completed in {time.time() - start_time:.2f} seconds.")
    return np.array(data), np.array(labels)

In [9]:
# Load MRI data
X, y = load_and_preprocess_images('./Train/')

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define the CSA optimization search space for RandomForest parameters
search_space = [np.array([10, 5]), np.array([100, 20])]

# Initialize CSA
csa = CrowSearchAlgorithm(num_crows=10, max_iter=5, search_space=search_space)

# Optimize parameters using CSA
print("Optimizing model parameters with CSA...")
best_solution, best_fitness = csa.optimize(X_train, y_train, X_val, y_val)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)

# Train final model with best parameters from CSA
print("Training the final model with optimized parameters...")
final_model = RandomForestClassifier(n_estimators=int(best_solution[0]), max_depth=int(best_solution[1]), random_state=42)
final_model.fit(X_train, y_train)


# Evaluate on test data
print("Evaluating model on test data...")
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")


Loading and preprocessing images...
Processing images for class: Non Demented
Processing images for class: Very mild Dementia
Processing images for class: Mild Dementia
Processing images for class: Moderate Dementia
Image loading and preprocessing completed in 33.34 seconds.
Optimizing model parameters with CSA...
Starting Crow Search Optimization...
Current params: [69.66174415 16.55722674], Validation accuracy: 1.0000
Iteration 1/5
Current params: [69.66174415 16.55722674], Validation accuracy: 1.0000
Current params: [69.66174415 16.55722674], Validation accuracy: 1.0000
Current params: [12.87163091 11.52077164], Validation accuracy: 1.0000
Current params: [40.81448782 18.74789228], Validation accuracy: 1.0000
Current params: [97.49069965 12.24619771], Validation accuracy: 1.0000
Current params: [26.84305937 15.13433196], Validation accuracy: 1.0000
Current params: [75.4703441  5.       ], Validation accuracy: 1.0000
Current params: [57.43920512 19.44151141], Validation accuracy: 1.0

In [12]:
# Save the trained model and scaler
model_path = "final_model.pkl"
scaler_path = "scaler.pkl"

print("Saving the model and scaler...")
joblib.dump(final_model, model_path)
joblib.dump(scaler, scaler_path)
print(f"Model saved to {model_path} and scaler saved to {scaler_path}")

Saving the model and scaler...
Model saved to final_model.pkl and scaler saved to scaler.pkl


In [ ]:
def predict_dementia_risk(image_path, model, scaler):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    img = img.flatten()
    img = scaler.transform([img])

    prob = model.predict_proba(img)[0]  # Probability for each class
    class_labels = ["Non Demented", "Very mild Dementia", "Mild Dementia", "Moderate Dementia"]
    most_likely_class = class_labels[np.argmax(prob)]
    likelihood = np.max(prob) * 100

    print(f"Class probabilities: {dict(zip(class_labels, prob * 100))}")
    return most_likely_class, likelihood

# Example usage
image_path = './New Data/Very mild Dementia/OAS1_0380_MR1_mpr-4_149.jpg'
most_likely_class, likelihood = predict_dementia_risk(image_path, final_model, scaler)
print(f"The most likely class is '{most_likely_class}' with a probability of {likelihood:.2f}%")

Class probabilities: {'Non Demented': np.float64(19.0), 'Very mild Dementia': np.float64(44.0), 'Mild Dementia': np.float64(21.0), 'Moderate Dementia': np.float64(16.0)}
The most likely class is 'Very mild Dementia' with a probability of 44.00%
